In [1]:
import openai
import panel as pn 
import json
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# from decouple import config

In [2]:
openai.api_key = "sk-"

In [3]:
def get_completion_from_messages(messages, model="gpt-4", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,          
    )

    return response.choices[0].message["content"]

In [4]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    
    context.append({'role':'assistant', 'content':f"{response}"})
    
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [5]:
pn.extension()

panels = [] 

context = [
    {
        'role': 'system',
        'content': """
        You are DataBot, an automated service designed to collect user information for a registration process. \
        Your goal is to engage users in a friendly and persuasive conversation to gather the required details. \
        Users may not answer correctly, may ask questions before deciding to answer, \
        or refuse to provide information until they are convinced about the reason. \
        Be a friendly persuader and ensure a natural conversation flow.

        You have multiple roles:
        - "Introduction Agent": Greet the user and explain the registration process.
        - "Information Gathering Agent": Ask for specific details such as name, email, phone number, address, date of birth, and education.
        - "Validation Agent": Persuade users to provide accurate information and address their concerns.
        - "Formatting Agent": Prepare the collected data for storage.
        
        **Important**: Before proceeding, please note that you have to perform validation checks on the information provided by user, including email and phone number formats. 
        Please ensure to clarify all options and details to uniquely identify each piece of information.

        You respond in a short, very conversational, and friendly style.

        Let's start the registration process!
        """
    }
]


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [6]:
messages =  context.copy()
messages.append(
{"role":"system", 'content':"Create a JSON summary of the user's information. Include the following fields:\
1) Name\
2) Email\
3) Phone Number\
4) Address\
5) Date of Birth\
6) Education\
Ensure that the information is accurately formatted in JSON, and summarize it for the system."},    
)   

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "Name": "Vivek Reddy",
  "Email": "vivekr12@gmail.com",
  "Phone Number": "+91-8872412949",
  "Address": "4/17, 3rd Cross Road, HSR Layout, Bangalore, 560103",
  "Date of Birth": "11-02-1999",
  "Education": "BE CSE"
}


In [19]:
data_dict = json.loads(response)

df = pd.DataFrame([data_dict])
df

,Name,Email,Phone Number,Address,Date of Birth,Education
0,Vivek Reddy,vivekr12@gmail.com,+91-8872412949,"4/17, 3rd Cross Road, HSR Layout, Bangalore, 5...",11-02-1999,BE CSE


In [20]:
df.to_csv("data_agent.csv", index=False)